In [ ]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.trainer_utils import get_last_checkpoint

# latest = get_last_checkpoint("fpf-7b-instructions-16k-call")

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained(latest)
# model = AutoModelForCausalLM.from_pretrained(
#     latest, 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama-7b-hf-2048-fpf', trust_remote_code=True)
kwargs = {'device_map': 'auto', 'torch_dtype': torch.float16}
model= AutoModelForCausalLM.from_pretrained('mesolitica/llama-7b-hf-2048-fpf', **kwargs)

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.29s/it]


In [3]:
from tqdm import tqdm
import os
import random
torch.manual_seed(404)
random.seed(404)

In [4]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/fizik-spm/data/'
with open(data_folder+ 'fizik') as fopen:
    text = fopen.read()
    
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    print(data)
    questions.append(data)

{'no': '1', 'soalan': 'Antara yang berikut, yang manakah kuantiti vektor?\nA. Tenaga\nB. Jisim\nC. Daya\nD. Laju', 'jawapan': 'C'}
{'no': '2', 'soalan': 'Manakah antara berikut mempunyai inersia yang paling kecil?\nA. 1.8 kg arnab\nB. 850 g spanar\nC. 1350 g kuali\nD. 1.25 kg mikroskop', 'jawapan': 'B'}
{'no': '3', 'soalan': 'Apakah konsep yang digunakan dalam pengukuran suhu kopi menggunakan termometer?\nA. Perolakan terma\nB. Muatan haba tentu\nC. Haba pendam tentu\nD. Keseimbangan terma', 'jawapan': 'D'}
{'no': '4', 'soalan': 'Antara hukum yang berikut, manakah menyatakan hubungan antara tekanan dan suhu pada isi padu gas malar?\nA. Hukum Boyle\nB. Hukum Charles\nC. Hukum Gay-Lussac\nD. Hukum Avogadro', 'jawapan': 'C'}
{'no': '5', 'soalan': 'Antara berikut, yang manakah merupakan ciri gelombang biasan dari kawasan air cetek ke kawasan air dalam?\nA. Sudut tuju lebih daripada sudut pembiasan.\nB. Panjang gelombang meningkat.\nC. Frekuensi berkurang.\nD. Frekuensi meningkat.', 'jawapa

In [5]:
arange = set(range(len(questions)))

In [6]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [7]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [8]:
for i in tqdm(range(len(questions))):
    
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_llama_chat(messages)
    print(prompt)
    print('----------------------------------------------------------------------')
    # break
    # inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    # repeat = []
    # for _ in range(5):
    #     try:
    #         generate_kwargs = dict(
    #             inputs,
    #             max_new_tokens=100,
    #             top_p=0.95,
    #             top_k=50,
    #             temperature=0.3,
    #             do_sample=True,
    #             num_beams=1,
    #             repetition_penalty=1.05,
    #         )
    #         r = model.generate(**generate_kwargs)
    #         splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
    #         splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
    #         repeat.append(splitted[0])
    
    #     except Exception as e:
    #         pass
    
    # questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_9774/667146085.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  shots = random.sample(arange - {i}, 1)
100%|████████████████████████████████████████| 20/20 [00:00<00:00, 32883.61it/s]

<s>[INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

Contoh soalan 1
soalan: Antara hukum yang berikut, manakah menyatakan hubungan antara tekanan dan suhu pada isi padu gas malar?
A. Hukum Boyle
B. Hukum Charles
C. Hukum Gay-Lussac
D. Hukum Avogadro
jawapan: C

soalan: Antara yang berikut, yang manakah kuantiti vektor?
A. Tenaga
B. Jisim
C. Daya
D. Laju
jawapan: [/INST]
----------------------------------------------------------------------
<s>[INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

Contoh soalan 1
soalan: Manakah kuantiti fizikal yang akan berubah apabila amplitud gelombang bertambah?
A. Tempoh berkurang
B. Tenaga bertambah
C. Laju akan bertambah
D. Panjang gelombang bertambah
jawapan: B

soalan: Manakah antara berikut mempunyai inersia yang paling kecil?
A. 1.8 kg arnab
B. 850 g spanar
C. 1350 g kuali
D. 1.25 kg mikroskop
jawapan: [/INST]
----------------------------------------------------------------------
<s>[INST] <<SYS>>
awak a

In [ ]:
print(prompt)

In [ ]:
def most_common(l):
    return max(set(l), key=l.count)

In [ ]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

In [ ]:
repeat

In [ ]:
for i in range(len(questions)):
    print(questions[i]['output'])